# 本チャプターの目次
1. テクニカルメタデータについて
2. テクニカルメタデータの一種データプロファイリング
3. PySparkでデータプロファイリングをしてみよう1 
4. PySparkでデータプロファイリングをしてみようその２
5. PySparkでデータプロファイリングをしてみようその3
6. データプロファイリングの結果をテーブルに格納してみよう
7. データ品質

# テクニカルメタデータについて

データマネジメントとは、

# テクニカルメタデータの一種データプロファイリング


# テクニカルメタデータの一種データプロファイリング

# PySparkでデータプロファイリングをしてみようその２

# PySparkでデータプロファイリングをしてみようその3

# データプロファイリングの結果をテーブルに格納してみよう



# データ品質

# 次セクションへの導入
次のセクションよりメタデータを解説及び、ところどころ算出していきます。
そのための準備を行います。

## データの読み込みとテーブルの作成をします※
1. jinko.csvを読み込んで表示する
2. jinko.csvの内容を保存するテーブルを作成する
3. jinko.csvのデータを2で作成したテーブルにInsertする

※以下の講座で行った最初の準備の簡略版です。
「データサイエンスのための前処理入門PythonとSparkで学ぶビッグデータエンジニアリング(PySpark) 速習講座」
https://www.udemy.com/course/python-spark-pyspark/?referralCode=E67BF8B61F65866794EB


In [ ]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init()

In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .enableHiveSupport() \
    .getOrCreate()


# spark.xxxxxと記載することで処理を分散させることが可能です。

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col

struct = StructType([
    StructField("code", StringType(), False),
    StructField("kenmei", StringType(), False),
    StructField("gengo", StringType(), False),
    StructField("wareki", StringType(), False),
    StructField("seireki", StringType(), False),
    StructField("chu", StringType(), False),
    StructField("sokei", StringType(), False),
    StructField("jinko_male", StringType(), False),
    StructField("jinko_female", StringType(), False)
])
df=spark.read.option("multiLine", "true").option("encoding", "SJIS") \
    .csv("/Users/saitouyuuki/Desktop/src/pyspark_dataprofiling_dataquality/dataset/jinko.csv", header=False, sep=',', inferSchema=False,schema=struct)
df.show()

In [ ]:
spark.sql(""" 
create database data_management_cruch_course;
""")

In [ ]:
# テーブルの作成をします。
# 今回はCSVに一致した「jinko_table」テーブルを作成します
spark.sql(""" 
CREATE EXTERNAL TABLE IF NOT EXISTS data_management_cruch_course.jinko_table (code String, gengo String,wareki String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
PARTITIONED BY (kenmei String)
STORED AS PARQUET
LOCATION '/Users/saitouyuuki/Desktop/src/pyspark_dataprofiling_dataquality/dataset/parquet/';
""")


In [ ]:
# 作成したテーブルの一覧を確認することができます
spark.sql("show tables in   data_management_cruch_course").show(truncate=False)

In [ ]:
# テーブルの定義を確認することができます。
spark.sql("show create table data_management_cruch_course.jinko_table").show(truncate=False)